In [1]:
import os
import sys
import argparse
import torch
from getdist import plots, MCSamples
import getdist
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import multivariate_normal

In [2]:
path = os.path.realpath(os.path.join(os.getcwd(), '../..'))
sys.path.insert(0, path)

In [3]:
from nnest.trainer import Trainer

SyntaxError: invalid syntax (networks.py, line 207)

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [ ]:
# Himmelblau
dims = 2
def loglike(z):
    z1 = z[:, 0]
    z2 = z[:, 1]
    return - (z1 ** 2 + z2 - 11.) ** 2 - (z1 + z2 ** 2 - 7.) ** 2
def transform(x):
    return 5. * x

In [ ]:
# Rosenbrock
#dims = 2
#def loglike(z):
#    return np.array([-sum(100.0 * (x[1:] - x[:-1] ** 2.0) ** 2.0 + (1 - x[:-1]) ** 2.0) for x in z])
#def transform(x):
#    return 5. * x

In [ ]:
# Gauss
#dims = 2
#corr = 0.99
#def loglike(x):
#    return multivariate_normal.logpdf(x, mean=np.zeros(dims), cov=np.eye(dims) + corr * (1 - np.eye(dims)))
#def transform(x):
#    return 3. * x

In [ ]:
# Eggbox
def loglike(x):
    chi = (np.cos(x[:,0] / 2.))*(np.cos(x[:,1] / 2.))
    return (2. + chi)**5
def transform(x):
    return x * 5 * np.pi

In [ ]:
n_samples = 1000
fraction = 0.02

In [ ]:
np.random.seed(0)
x = 2 * (np.random.uniform(size=(int(n_samples / fraction), 2)) - 0.5)
likes = loglike(transform(x))
idx = np.argsort(-likes)
samples = x[idx[0:n_samples]]

In [ ]:
t = Trainer(dims, 64,  num_blocks=5, num_layers=1, scale='constant', flow='nvp')

In [ ]:
t.train(samples, jitter=0.01)

In [ ]:
x_synth = t.netG.sample(samples.size).detach().cpu().numpy()
z = t.get_latent_samples(samples)

In [ ]:
grid = []
for x in np.linspace(np.min(samples[:, 0])*1.2, np.max(samples[:, 0])*1.2, 10):
    for y in np.linspace(np.min(samples[:, 1])*1.2, np.max(samples[:, 1])*1.2, 5000):
        grid.append([x, y])
for y in np.linspace(np.min(samples[:, 1])*1.2, np.max(samples[:, 1])*1.2, 10):
    for x in np.linspace(np.min(samples[:, 0])*1.2, np.max(samples[:, 0])*1.2, 5000):
        grid.append([x, y])
grid = np.array(grid)
z_grid = t.get_latent_samples(grid)

In [ ]:
fig, ax = plt.subplots(1, 3, figsize=(12, 5))
ax[0].scatter(grid[:, 0], grid[:, 1], c=grid[:, 0], marker='.', s=1, linewidths=0)
ax[0].scatter(samples[:, 0], samples[:, 1], s=4)
ax[0].set_title('Real data')
ax[1].scatter(z_grid[:, 0], z_grid[:, 1], c=grid[:, 0], marker='.', s=1, linewidths=0)
ax[1].scatter(z[:, 0], z[:, 1], s=4)
ax[1].set_title('Latent data')
ax[1].set_xlim([-np.max(np.abs(z)), np.max(np.abs(z))])
ax[1].set_ylim([-np.max(np.abs(z)), np.max(np.abs(z))])
ax[2].scatter(x_synth[:, 0], x_synth[:, 1], s=2)
ax[2].set_title('Synthetic data')
plt.tight_layout()
plt.show()